# Investigating WeRateDogs‘s Twitter account
by Emadeldin Fawaz

## Installing needed packages and tweepy

In [2]:
import tweepy
import pandas as pd
import numpy as np
import requests
import json
import time
import datetime as dt
import matplotlib.pyplot as plt

# 1. Data Gathering

In [3]:
# Supplied file
tw_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [5]:
# Requesting Tweet image predictions

with open('image_predictions.tsv' , 'wb') as file:
    image_predictions = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv', auth=('user', 'pass'))
    file.write(image_predictions.content)   

In [6]:
# Reading image predictions
image_predictions = pd.read_csv('image_predictions.tsv', sep='\t')

## Testing the data gathering process so far

In [7]:
tw_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [8]:
tw_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [9]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [10]:
image_predictions.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1511,786233965241827333,https://pbs.twimg.com/media/CulDnZpWcAAGbZ-.jpg,1,Labrador_retriever,0.478193,True,schipperke,0.224817,True,Staffordshire_bullterrier,0.077396,True
1603,800141422401830912,https://pbs.twimg.com/media/CxqsX-8XUAAEvjD.jpg,3,golden_retriever,0.938048,True,kuvasz,0.025119,True,Labrador_retriever,0.022977,True
624,680609293079592961,https://pbs.twimg.com/media/CXICiB9UwAE1sKY.jpg,1,French_bulldog,0.700764,True,Chihuahua,0.072390,True,American_Staffordshire_terrier,0.039619,True
1644,808106460588765185,https://pbs.twimg.com/media/Czb4iFRXgAIUMiN.jpg,1,golden_retriever,0.426183,True,Labrador_retriever,0.257447,True,Great_Pyrenees,0.126482,True
225,670338931251150849,https://pbs.twimg.com/media/CU2FsRnVAAA3TEg.jpg,1,cairn,0.245033,True,West_Highland_white_terrier,0.137709,True,miniature_schnauzer,0.089173,True


In [11]:
# Tweepy keys and token

# CONSUMER_KEY = 'Consumer Key (API key)'
# CONSUMER_SECRET = 'Consumer Secret (API Secret)'
# OAUTH_TOKEN = 'Access Token'
# OAUTH_TOKEN_SECRET = 'Access Token Secret'

consumer_key = 'xxxxconsumer_secret = 'xxxxaccess_token = 'xxxxxaccess_secret = 'xxxxxxx
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# Construct the API instance
api = tweepy.API(auth, parser = tweepy.parsers.JSONParser(), wait_on_rate_limit = True, 
                 wait_on_rate_limit_notify = True) 

# Source of the Tweet API Code: Anouar ZBAIDA 

In [16]:
# Liste where we will store the dictionaries of our result
df_list = []
# Liste frame where we will store the tweet_id of the errors
error_list = []

# Calculate the time of excution
start = time.time()

# Get the tweet object for all the teweets in archive dataframe 
for tweet_id in tw_archive['tweet_id']:
    try:
        page = api.get_status(tweet_id, tweet_mode = 'extended')
        # Print one page to look at the structure of the returned file
        # and the names of attributes
        # print(json.dumps(page, indent = 4))
        #break
        
        favorites = page['favorite_count'] # How many favorites the tweet had
        retweets = page['retweet_count'] # Count of the retweet
        user_followers = page['user']['followers_count'] # How many followers the user had
        user_favourites = page['user']['favourites_count'] # How many favorites the user had
        date_time = page['created_at'] # The date and time of the creation
        
        df_list.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets),
                        'user_followers': int(user_followers),
                        'user_favourites': int(user_favourites),
                        'date_time': pd.to_datetime(date_time)})
    
    # Catch the exceptions of the TweepError
    except Exception as e:
        print(str(tweet_id)+ " _ " + str(e))
        error_list.append(tweet_id)

# Calculate the time of excution
end = time.time()
print(end - start)

888202515573088257 _ [{'code': 144, 'message': 'No status found with that ID.'}]
873697596434513921 _ [{'code': 144, 'message': 'No status found with that ID.'}]
872668790621863937 _ [{'code': 144, 'message': 'No status found with that ID.'}]
869988702071779329 _ [{'code': 144, 'message': 'No status found with that ID.'}]
866816280283807744 _ [{'code': 144, 'message': 'No status found with that ID.'}]
861769973181624320 _ [{'code': 144, 'message': 'No status found with that ID.'}]
845459076796616705 _ [{'code': 144, 'message': 'No status found with that ID.'}]
842892208864923648 _ [{'code': 144, 'message': 'No status found with that ID.'}]
837012587749474308 _ [{'code': 144, 'message': 'No status found with that ID.'}]
827228250799742977 _ [{'code': 144, 'message': 'No status found with that ID.'}]
802247111496568832 _ [{'code': 144, 'message': 'No status found with that ID.'}]
Rate limit reached. Sleeping for: 571
775096608509886464 _ [{'code': 144, 'message': 'No status found with th

In [17]:
# We repeat the same operation for the tweet_ids that we coudln't get and append the result to df_list
ee_list = []
for e in error_list:
    try:
        favorites = page['favorite_count']
        retweets = page['retweet_count']
        user_followers = page['user']['followers_count']
        user_favourites = page['user']['favourites_count']
        date_time = page['created_at']
        
        df_list.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets),
                        'user_followers': int(user_followers),
                        'user_favourites': int(user_favourites),
                        'date_time': pd.to_datetime(date_time)})
        
    except Exception:
        print(str(tweet_id)+ " _ " + str(e))
        ee_list.append(e)

In [18]:
# Create DataFrames from list of dictionaries
tweet_status = pd.DataFrame(df_list, columns = ['tweet_id', 'favorites', 'retweets',
                                               'user_followers', 'user_favourites', 'date_time'])

# Save the dataFrame in file
tweet_status.to_csv('tweet_json.txt', encoding = 'utf-8', index=False)

In [21]:
tweet_status = pd.read_csv('tweet_json.txt', encoding = 'utf-8')
tweet_status.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 6 columns):
tweet_id           2356 non-null int64
favorites          2356 non-null int64
retweets           2356 non-null int64
user_followers     2356 non-null int64
user_favourites    2356 non-null int64
date_time          2356 non-null object
dtypes: int64(5), object(1)
memory usage: 110.5+ KB


In [22]:
tweet_status.sample(5)

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
1237,710997087345876993,4774,1488,7461358,140021,2016-03-19 01:11:29
1515,690248561355657216,1773,438,7461358,140021,2016-01-21 19:04:15
1982,672591762242805761,968,345,7462017,140024,2015-12-04 01:42:26
348,830173239259324417,0,6193,7461337,140021,2017-02-10 21:54:58
1021,744995568523612160,3134,671,7461354,140021,2016-06-20 20:49:19


## Data Assessment

In [24]:
# Reading gathered files
tw_archive = pd.read_csv('twitter-archive-enhanced.csv')
image_predictions = pd.read_csv('image_predictions.tsv', sep='\t')
tweet_status = pd.read_csv('tweet_json.txt', encoding = 'utf-8')

### Visual assessment

In [25]:
tw_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [26]:
image_predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [27]:
tweet_status

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
0,892420643555336193,38186,8364,7461335,140021,2017-08-01 16:23:56
1,892177421306343426,32756,6176,7461335,140021,2017-08-01 00:17:27
2,891815181378084864,24661,4088,7461335,140021,2017-07-31 00:18:03
3,891689557279858688,41517,8503,7461335,140021,2017-07-30 15:58:51
4,891327558926688256,39707,9201,7461335,140021,2017-07-29 16:00:24
5,891087950875897856,19932,3063,7461335,140021,2017-07-29 00:08:17
6,890971913173991426,11662,2028,7461335,140021,2017-07-28 16:27:12
7,890729181411237888,64434,18557,7461335,140021,2017-07-28 00:22:40
8,890609185150312448,27387,4204,7461335,140021,2017-07-27 16:25:51
9,890240255349198849,31435,7264,7461335,140021,2017-07-26 15:59:51


### Programmatic assessment

### Assessing the Archive document

In [28]:
# Check for anomalies (min or max in comparison to the mnean and the 5 digits description (median and percentiles))
tw_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [29]:
# Check for missing values and if column types are correct as needed
tw_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [55]:
# Check for duplicates
assert [tw_archive.duplicated() == False]

In [30]:
# Find the row(s) where the denominator = 0. This must be a mistake.
tw_archive[tw_archive['rating_denominator'] == 0 ]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None


In [31]:
# Read the text in the tweet with 0 denominator to evaluate what has gone wrong (if possible)
tw_archive['text'][tw_archive['rating_denominator'] == 0 ]

313    @jonnysun @Lin_Manuel ok jomny I know you're e...
Name: text, dtype: object

Note: delete this row during cleaning.

In [32]:
# Denominators must be 10 (by defacto), check rows with denominators less than 10
tw_archive[tw_archive['rating_denominator'] < 10 ]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,26259576.0,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
2335,666287406224695296,NaN,NaN,2015-11-16 16:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an Albanian 3 1/2 legged Episcopalian...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666287406...,1,2,an,None,None,None,None


In [40]:
# Denominators must be 10 (by defacto), check rows with denominators more than 20
tw_archive[tw_archive['rating_denominator'] > 20 ]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to this unbelievably well behaved sq...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/731156023...,204,170,this,None,None,None,None
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bluebert. He just saw that both #Final...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/716439118...,50,50,Bluebert,None,None,None,None
1228,713900603437621249,NaN,NaN,2016-03-27 01:29:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Saturday here's 9 puppers on a bench. 99...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/713900603...,99,90,None,None,None,None,None
1254,710658690886586372,NaN,NaN,2016-03-18 02:46:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a brigade of puppers. All look very pre...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/710658690...,80,80,None,None,None,None,None
1274,709198395643068416,NaN,NaN,2016-03-14 02:04:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...","From left to right:\nCletus, Jerome, Alejandro...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/709198395...,45,50,None,None,None,None,None
1351,704054845121142784,NaN,NaN,2016-02-28 21:25:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a whole flock of puppers. 60/50 I'll ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/704054845...,60,50,a,None,None,None,None
1433,697463031882764288,NaN,NaN,2016-02-10 16:51:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Happy Wednesday here's a bucket of pups. 44/40...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/697463031...,44,40,None,None,None,None,None
1634,684225744407494656,6.842229e+17,4.196984e+09,2016-01-05 04:11:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Two sneaky puppers were not initially seen, mo...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/684225744...,143,130,None,None,None,None,None


In [33]:
# Numerators must be more than 10 (this is what is unique about this WeRateDog). Check rows with 0 values in numerator
tw_archive[tw_archive['rating_numerator'] == 0 ]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
315,835152434251116546,NaN,NaN,2017-02-24 15:40:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you're so blinded by your systematic plag...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/835152434...,0,10,None,None,None,None,None
1016,746906459439529985,7.468859e+17,4.196984e+09,2016-06-26 03:22:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...","PUPDATE: can't see any. Even if I could, I cou...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/746906459...,0,10,None,None,None,None,None


In [34]:
# Double check that it is only two rows
len(tw_archive[tw_archive['rating_numerator'] == 0 ])

2

In [35]:
# Check anomalies in the numerator (very large values or values way more than the average)
tw_archive[tw_archive['rating_numerator'] > 20]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,NaN,NaN,NaN,666,10,None,None,None,None,None
290,838150277551247360,8.381455e+17,2.195506e+07,2017-03-04 22:12:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@markhoppus 182/10,NaN,NaN,NaN,NaN,182,10,None,None,None,None,None
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
340,832215909146226688,NaN,NaN,2017-02-16 13:11:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: This is Logan, the Chow who liv...",7.867091e+17,4.196984e+09,2016-10-13 23:23:56 +0000,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
695,786709082849828864,NaN,NaN,2016-10-13 23:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Logan, the Chow who lived. He solemnly...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None
763,778027034220126208,NaN,NaN,2016-09-20 00:24:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sophie. She's a Jubilant Bush Pupper. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/778027034...,27,10,Sophie,None,None,pupper,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None


In [36]:
# Count rows with high numerator (more than 20)
len(tw_archive[tw_archive['rating_numerator'] > 20 ])

24

There are 24 rows with anomalies in the numerator (>20). Deleting these rows will enhance the statestics (such as median) and data visualization (by inimizing outliers). However, before deleting all of them, check if the values are wrong or could be fixed.


As the Denominator is fixed to be 10, capturing strange values like 40, 50 or 80 indicates a mistake in data entry (probably by the Twitter Account holder him/herself). This might be even more illustrated when the numerator also has the value of 44, 40, 50, 80, 88. This indicates that a typo mistake happened here where a 4, 5 or 8 was mixed with an 1.

### Assissing the Image Document

In [37]:
# Check for missing values and column types
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [38]:
# Check for anomalies and the basic descriptive statistics
image_predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [56]:
# Check fr duplicates
assert [image_predictions.duplicated() == False]

In [44]:
# Check the values in the image predictions and if there are any an usual value
image_predictions.p1.value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
cocker_spaniel                30
malamute                      30
French_bulldog                26
Chesapeake_Bay_retriever      23
miniature_pinscher            23
seat_belt                     22
German_shepherd               20
Siberian_husky                20
Staffordshire_bullterrier     20
web_site                      19
Cardigan                      19
Maltese_dog                   18
teddy                         18
beagle                        18
Eskimo_dog                    18
Shetland_sheepdog             18
Lakeland_terrier              17
Rottweiler                    17
Shih-Tzu                      17
kuvasz                        16
Italian_greyhound             16
          

In [45]:
# Find how many rare things (frequency less than 3) appeared in the data and investigaate what they are
rare_things= image_predictions.groupby('p1').filter(lambda x: len(x) < 3)

In [46]:
rare_things.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
28,666407126856765440,https://pbs.twimg.com/media/CT-NvwmW4AAugGZ.jpg,1,black-and-tan_coonhound,0.529139,True,bloodhound,0.244220,True,flat-coated_retriever,0.173810,True
1970,868880397819494401,https://pbs.twimg.com/media/DA7iHL5U0AA1OQo.jpg,1,laptop,0.153718,False,French_bulldog,0.099984,True,printer,0.077130,False
929,702932127499816960,https://pbs.twimg.com/media/CcFRCfRW4AA5a72.jpg,1,wallaby,0.410710,False,wombat,0.239332,False,beaver,0.149605,False
1856,841439858740625411,https://pbs.twimg.com/media/C61lFFiWoAAJdiL.jpg,3,military_uniform,0.853684,False,Labrador_retriever,0.048200,True,groenendael,0.015394,True
1755,824796380199809024,https://pbs.twimg.com/media/CwiuEJmW8AAZnit.jpg,2,gas_pump,0.676439,False,harvester,0.049995,False,swing,0.044660,False


In [47]:
len(rare_things)

271

In [16]:
len(image_predictions)

2075

### Assessing Tweet Status


In [58]:
# Check for missing values and column types
tweet_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 6 columns):
tweet_id           2356 non-null int64
favorites          2356 non-null int64
retweets           2356 non-null int64
user_followers     2356 non-null int64
user_favourites    2356 non-null int64
date_time          2356 non-null object
dtypes: int64(5), object(1)
memory usage: 110.5+ KB


In [60]:
# Check for duplicates
assert [tweet_status.tweet_id.duplicated() == False]

In [61]:
# Check for anomalies
tweet_status.describe()

,tweet_id,favorites,retweets,user_followers,user_favourites
count,2.356000e+03,2356.000000,2356.000000,2.356000e+03,2356.000000
mean,7.417684e+17,7962.654499,2933.644737,7.461371e+06,140021.024194
std,6.837209e+16,12345.699353,4947.190933,6.167252e+01,0.268376
min,6.660209e+17,0.000000,0.000000,7.461335e+06,140021.000000
25%,6.776996e+17,1398.500000,583.000000,7.461340e+06,140021.000000
50%,7.178159e+17,3461.500000,1365.000000,7.461357e+06,140021.000000
75%,7.986755e+17,9779.750000,3417.250000,7.461400e+06,140021.000000
max,8.924206e+17,164825.000000,84021.000000,7.462017e+06,140024.000000


### Assessment report

<strong>Tidiness Issues (structural issues)</strong>

<strong>Tw_archive</strong>

- Not all columns are needed for further analysis

- Replace 'None' with np.nan to indicate the missing values

-	The columns (doggo, floofer, pupper and puppo) do not need to be separated. Each dog will be classified as one of these classifications. It is better to create one column for dog classification that contains the values (doggo, floofer, pupper and puppo).



<b>image_predictions</b>

-   Change the names of columns type (p1, p2 and p3) to a more descriptive name. 
-	The P1 confidence interval might not be always convincing. I will set the acceptance threshold at P ≥ 0.5. This might mean neglecting a lot of the data (the median is .585). However, the trade off is higher confidence in the results.


<b>tweet_status</b>

- The two columns user_followers & user_favourites contain fixed values and will not be needed in the analysis.




<strong>Quality issues</strong>

<b>Tw_archive</b>

-	Some columns like (in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id) are missing too many values and might not be needed in this analysis.
-	For the column “rating_numirator” there are zero values. delete these two rows with zero in the numirator.
-	For the column “rating_numirator” there are extreme values. 
-	Delete the cells with numerator more than 20. This will enhance visualization (by reducing outliers) and statistics. 
-	For the column “rating_denumirator” there is a zero value. delete it. 
-	For the column “rating_denumirator” there are extreme values. 
-	Correcting anomalies and typo mistakes in the numerator and the denominator:
As the Denominator is fixed to be 10, capturing strange values like 40, 50 or 80 indicates a mistake in data entry (probably by the Twitter Account holder him/herself). This might be even more illustrated when the numerator also has the value of 44, 40, 50, 80, 88. This indicates that a typo mistake happened here where a 4, 5 or 8 was mixed with an 1.
-	Line  1202‎	numerator must be 10 not 50, and denominator must be 10 (typo 5 instead of 1)‎
-	Line  1254‎	numerator must be 10 not 80, and denominator must be 10 (typo 8 ‎instead of 1)‎
-	Line  1433‎	numerator must be 11 not 44, and denominator must be 10 (typo 4 ‎instead of 1)‎
-	Line  1843‎	numerator must be 11 not 88, and denominator must be 10 (typo 8 ‎instead of 1)‎
-	It is not clear which typo mistake is it with the rest of the extreme denominator ‎values.‎



<b>image_predictions</b>

- Drop unneeded columns
- “Egyptian_cat”, is that a dog? Delete this row and rows with similar strange values like for eg. "Laptop".
- There are some strange values in the p1 columns such as “school_bus, pillow, cartoon” ‎these cells must be checked first then deleted if not needed. 
- 271 rows (with object frequency less than 3) could be deleted because they are not that frequent and most probably not dogs. This will solve the problems in the previous two points

<b>tweet_status</b>

-	Drop unneeded columns



<strong>Mereg all documents usig tweet_id</strong>

## Data Cleaning

## Cleaning tw_archive

In [62]:
# Reading gathered files
tw_archive = pd.read_csv('twitter-archive-enhanced.csv')
image_predictions = pd.read_csv('image_predictions.tsv', sep='\t')
tweet_status = pd.read_csv('tweet_json.txt', encoding = 'utf-8')

In [63]:
# create a cleaned dataframe from the archive
cleaned_archive = tw_archive.copy()

In [64]:
# test
cleaned_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

The column "retweeted_status_id" is showing if the tweet was original or a reply to another tweet.
Select only rows where this column has a NaN value. This should reveal the original tweets and delete all retweets.


In [65]:
# Define: Select rows wehre the column "in_reply_to_status_id" is Null (this will exclude all the retweets)

#Code
cleaned_archive = cleaned_archive[cleaned_archive['retweeted_status_id'].isnull()]

In [66]:
# test
cleaned_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2175 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2175 non-null object
source                        2175 non-null object
text                          2175 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2117 non-null object
rating_numerator              2175 non-null int64
rating_denominator            2175 non-null int64
name                          2175 non-null object
doggo                         2175 non-null object
floofer                       2175 non-null object
pupper                        2175 non-null object
puppo                         2175 non-null object
dtypes: float64(4), int64(3), object(1

In [67]:
# Define: Dropping unneded (missing too much values) columns using drop function.

# code
cleaned_archive = cleaned_archive.drop(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1)

In [68]:
# Test
cleaned_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null object
source                2175 non-null object
text                  2175 non-null object
expanded_urls         2117 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: int64(3), object(9)
memory usage: 220.9+ KB


In [69]:
# Define: I do not need 'expanded_urls' in my analysis. use drop function
# Code
cleaned_archive = cleaned_archive.drop(['expanded_urls'], axis = 1)

In [70]:
# Test
cleaned_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null object
source                2175 non-null object
text                  2175 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: int64(3), object(8)
memory usage: 203.9+ KB


In [75]:
# Define: use loc to correct the typo mistakes in specified cells

# Code

cleaned_archive.loc[cleaned_archive['rating_numerator']==50, ['rating_numerator']] = 10
cleaned_archive.loc[cleaned_archive['rating_denominator']==50, ['rating_denominator']] = 10

cleaned_archive.loc[cleaned_archive['rating_numerator']==88, ['rating_numerator']] = 11
cleaned_archive.loc[cleaned_archive['rating_denominator']==80, ['rating_denominator']] = 10

cleaned_archive.loc[cleaned_archive['rating_numerator']==80, ['rating_numerator']] = 10
cleaned_archive.loc[cleaned_archive['rating_denominator']==80, ['rating_denominator']] = 10

cleaned_archive.loc[cleaned_archive['rating_numerator']==44, ['rating_numerator']] = 11
cleaned_archive.loc[cleaned_archive['rating_denominator']==40, ['rating_denominator']] = 10



In [76]:
# test
cleaned_archive[cleaned_archive['rating_numerator'] > 20 ]

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
188,855862651834028034,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,420,10,None,None,None,None,None
189,855860136149123072,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,666,10,None,None,None,None,None
290,838150277551247360,2017-03-04 22:12:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@markhoppus 182/10,182,10,None,None,None,None,None
313,835246439529840640,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,960,0,None,None,None,None,None
433,820690176645140481,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,84,70,None,None,None,None,None
516,810984652412424192,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,24,7,Sam,None,None,None,None
695,786709082849828864,2016-10-13 23:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Logan, the Chow who lived. He solemnly...",75,10,Logan,None,None,None,None
763,778027034220126208,2016-09-20 00:24:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sophie. She's a Jubilant Bush Pupper. ...,27,10,Sophie,None,None,pupper,None
902,758467244762497024,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,165,150,None,None,None,None,None
979,749981277374128128,2016-07-04 15:00:45 +0000,"<a href=""https://about.twitter.com/products/tw...",This is Atticus. He's quite simply America af....,1776,10,Atticus,None,None,None,None


In [77]:
len(cleaned_archive[cleaned_archive['rating_numerator'] > 20 ])

19

In [78]:
cleaned_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2175 non-null int64
timestamp             2175 non-null object
source                2175 non-null object
text                  2175 non-null object
rating_numerator      2175 non-null int64
rating_denominator    2175 non-null int64
name                  2175 non-null object
doggo                 2175 non-null object
floofer               2175 non-null object
pupper                2175 non-null object
puppo                 2175 non-null object
dtypes: int64(3), object(8)
memory usage: 203.9+ KB


In [79]:
# Define: exclude all extreme and zero values from the numertor and denuminator

#Code
cleaned_archive = cleaned_archive[cleaned_archive['rating_numerator'] != 0 ]
cleaned_archive = cleaned_archive[cleaned_archive['rating_denominator'] >= 10 ]
cleaned_archive = cleaned_archive[cleaned_archive['rating_numerator'] <= 20 ]
cleaned_archive = cleaned_archive[cleaned_archive['rating_denominator'] < 20 ]


In [80]:
# test
len(cleaned_archive[cleaned_archive['rating_numerator'] > 20 ])

0

In [81]:
len(cleaned_archive[cleaned_archive['rating_denominator'] < 10 ])

0

In [82]:
len(cleaned_archive[cleaned_archive['rating_denominator'] > 20 ])

0

In [83]:
cleaned_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2151 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2151 non-null int64
timestamp             2151 non-null object
source                2151 non-null object
text                  2151 non-null object
rating_numerator      2151 non-null int64
rating_denominator    2151 non-null int64
name                  2151 non-null object
doggo                 2151 non-null object
floofer               2151 non-null object
pupper                2151 non-null object
puppo                 2151 non-null object
dtypes: int64(3), object(8)
memory usage: 201.7+ KB


In [84]:
# Define: replace the value 'None' with the NaN (missing value)

# Code:
cleaned_archive =  cleaned_archive.replace('None', np.nan)

In [85]:
# test
cleaned_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2151 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id              2151 non-null int64
timestamp             2151 non-null object
source                2151 non-null object
text                  2151 non-null object
rating_numerator      2151 non-null int64
rating_denominator    2151 non-null int64
name                  1488 non-null object
doggo                 87 non-null object
floofer               10 non-null object
pupper                233 non-null object
puppo                 25 non-null object
dtypes: int64(3), object(8)
memory usage: 201.7+ KB


In [86]:
# Define: use loc to add a new column dog_class = doggo, floofer, pupper or poppo. NaN will be used if not any of the previous

# Code
cleaned_archive.loc[cleaned_archive['doggo'] == 'doggo', 'dog_class'] = 'doggo'
cleaned_archive.loc[cleaned_archive['floofer'] == 'floofer', 'dog_class'] = 'floofer'
cleaned_archive.loc[cleaned_archive['pupper'] == 'pupper', 'dog_class'] = 'pupper'
cleaned_archive.loc[cleaned_archive['puppo'] == 'puppo', 'dog_class'] = 'puppo'



In [88]:
# Test
cleaned_archive.sample(10)

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,dog_class
1251,710997087345876993,2016-03-19 01:11:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Milo and Amos. They are the best of pals....,12,10,Milo,NaN,NaN,NaN,NaN,NaN
1655,683391852557561860,2016-01-02 20:58:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Say hello to Jack (pronounced ""Kevin""). He's a...",11,10,Jack,NaN,NaN,NaN,NaN,NaN
175,857989990357356544,2017-04-28 16:08:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rosie. She was just informed of the wa...,12,10,Rosie,NaN,NaN,NaN,NaN,NaN
1037,744709971296780288,2016-06-20 01:54:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Harvey. He's stealthy af. 10/10 would ...,10,10,Harvey,NaN,NaN,NaN,NaN,NaN
283,838952994649550848,2017-03-07 03:22:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",SHE MISPLACED HER HOOMAN 13/10 MISTAKES HAPPEN...,13,10,NaN,NaN,NaN,NaN,NaN,NaN
2323,666428276349472768,2015-11-17 01:30:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have an Austrian Pulitzer. Collectors ...,7,10,NaN,NaN,NaN,NaN,NaN,NaN
377,828011680017821696,2017-02-04 22:45:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Say hello to Brutus and Jersey. They think the...,11,10,Brutus,NaN,NaN,NaN,NaN,NaN
1567,687826841265172480,2016-01-15 02:41:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Louis. He's a rollercoaster of emotion...,12,10,Louis,NaN,NaN,NaN,NaN,NaN
2154,669625907762618368,2015-11-25 21:17:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Penny. She's having fun AND being safe...,12,10,Penny,NaN,NaN,NaN,NaN,NaN
1578,687317306314240000,2016-01-13 16:56:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tyrone. He's a leaf wizard. Self-motiv...,11,10,Tyrone,NaN,NaN,NaN,NaN,NaN


In [89]:
# test
cleaned_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2151 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2151 non-null int64
timestamp             2151 non-null object
source                2151 non-null object
text                  2151 non-null object
rating_numerator      2151 non-null int64
rating_denominator    2151 non-null int64
name                  1488 non-null object
doggo                 87 non-null object
floofer               10 non-null object
pupper                233 non-null object
puppo                 25 non-null object
dog_class             343 non-null object
dtypes: int64(3), object(9)
memory usage: 218.5+ KB


In [90]:
# dropping unneded doggo, floofer, pupper or poppo columns
cleaned_archive = cleaned_archive.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis = 1)

In [91]:
cleaned_archive.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2151 entries, 0 to 2355
Data columns (total 8 columns):
tweet_id              2151 non-null int64
timestamp             2151 non-null object
source                2151 non-null object
text                  2151 non-null object
rating_numerator      2151 non-null int64
rating_denominator    2151 non-null int64
name                  1488 non-null object
dog_class             343 non-null object
dtypes: int64(3), object(5)
memory usage: 151.2+ KB


### Cleaning images dataframe

In [92]:
# Create a working df 
cleaned_images = image_predictions.copy()

In [93]:
# Test
cleaned_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [94]:
# Define: rename columns 

# Code
cleaned_images= cleaned_images.rename(columns={'p1':'Breed_probability1', 'p2':'Breed_probability2', 'p3':'Breed_probability3'})


In [95]:
# Test
cleaned_images.sample(5)

,tweet_id,jpg_url,img_num,Breed_probability1,p1_conf,p1_dog,Breed_probability2,p2_conf,p2_dog,Breed_probability3,p3_conf,p3_dog
514,676237365392908289,https://pbs.twimg.com/media/CWJ6Sc-WwAAlpI6.jpg,1,French_bulldog,0.961996,True,Chihuahua,0.021793,True,Boston_bull,0.006916,True
1922,856543823941562368,https://pbs.twimg.com/media/C-MOEDCXYAEjp7o.jpg,1,Boston_bull,0.306910,True,Siamese_cat,0.191218,False,Chihuahua,0.189288,True
1087,718939241951195136,https://pbs.twimg.com/media/CfovbK4WIAAkTn3.jpg,1,Pembroke,0.766327,True,Cardigan,0.222126,True,toilet_tissue,0.006757,False
1365,761599872357261312,https://pbs.twimg.com/media/CpG_CrlWYAYyuP3.jpg,1,Gordon_setter,0.240427,True,Saluki,0.224269,True,Doberman,0.129730,True
818,692901601640583168,https://pbs.twimg.com/media/CZ2uU37UcAANzmK.jpg,1,soft-coated_wheaten_terrier,0.403496,True,cocker_spaniel,0.135164,True,golden_retriever,0.088719,True


In [96]:
# Define: Exclude all rows with P1_confidence less than 0.5

# Code: 
cleaned_images = cleaned_images[cleaned_images['p1_conf'] > 0.5 ]

In [97]:
# test
assert [cleaned_images[cleaned_images['p1_conf'] > 0.5 ]]

In [98]:
# Define: use filter to select only needed columns

# Code
cleaned_images = cleaned_images.filter(['tweet_id','Breed_probability1','p1_conf'] )



In [99]:
# test 
cleaned_images.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1239 entries, 1 to 2072
Data columns (total 3 columns):
tweet_id              1239 non-null int64
Breed_probability1    1239 non-null object
p1_conf               1239 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 38.7+ KB


In [100]:
# Define: use groupby to select not rare things in the data (frequency more than 3)

# Code
cleaned_images = cleaned_images.groupby('Breed_probability1').filter(lambda x: len(x) > 3)

In [101]:
# Test
cleaned_images.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 987 entries, 2 to 2072
Data columns (total 3 columns):
tweet_id              987 non-null int64
Breed_probability1    987 non-null object
p1_conf               987 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 30.8+ KB


## Cleaning tweet_status

In [102]:
#Create a working df
cleaned_status = tweet_status.copy()

In [103]:
cleaned_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 6 columns):
tweet_id           2356 non-null int64
favorites          2356 non-null int64
retweets           2356 non-null int64
user_followers     2356 non-null int64
user_favourites    2356 non-null int64
date_time          2356 non-null object
dtypes: int64(5), object(1)
memory usage: 110.5+ KB


In [104]:
# Define: filter/select needed columns
# Code
cleaned_status = cleaned_status.filter(['tweet_id','favorites','retweets', 'date_time'] )

In [105]:
# test
cleaned_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 4 columns):
tweet_id     2356 non-null int64
favorites    2356 non-null int64
retweets     2356 non-null int64
date_time    2356 non-null object
dtypes: int64(3), object(1)
memory usage: 73.7+ KB


## Merging documents to form a working dataframe

In [106]:
tweeter_df = pd.merge(cleaned_archive, cleaned_images, how='outer', on=['tweet_id'])

In [107]:
tweeter_df = pd.merge(tweeter_df, cleaned_status, how = 'outer', on=['tweet_id'])

In [108]:
# Test the merge
tweeter_df.sample(5)

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,dog_class,Breed_probability1,p1_conf,favorites,retweets,date_time
2072,667443425659232256,2015-11-19 20:44:47 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Exotic dog here. Long neck. Weird paws. Obsess...,6.0,10.0,NaN,NaN,NaN,NaN,791.0,591.0,2015-11-19 20:44:47
839,745789745784041472,2016-06-23 01:25:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Gus. He didn't win the Powerball. Quit...,10.0,10.0,Gus,NaN,Pekinese,0.984267,4249.0,1147.0,2016-06-23 01:25:06
882,739979191639244800,2016-06-07 00:36:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Nollie. She's waving at you. If you do...,12.0,10.0,Nollie,NaN,NaN,NaN,21009.0,6347.0,2016-06-07 00:36:02
574,783334639985389568,2016-10-04 15:55:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Dave. He's currently in a predicament....,12.0,10.0,Dave,NaN,Cardigan,0.593858,31366.0,12902.0,2016-10-04 15:55:06
59,880095782870896641,2017-06-28 16:09:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Please don't send in photos without dogs in th...,11.0,10.0,NaN,NaN,NaN,NaN,27265.0,4297.0,2017-06-28 16:09:20


In [109]:
tweeter_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2360 entries, 0 to 2359
Data columns (total 13 columns):
tweet_id              2360 non-null int64
timestamp             2165 non-null object
source                2165 non-null object
text                  2165 non-null object
rating_numerator      2165 non-null float64
rating_denominator    2165 non-null float64
name                  1488 non-null object
dog_class             343 non-null object
Breed_probability1    987 non-null object
p1_conf               987 non-null float64
favorites             2356 non-null float64
retweets              2356 non-null float64
date_time             2356 non-null object
dtypes: float64(5), int64(1), object(7)
memory usage: 258.1+ KB


There are some missing values in some columns. However, based on the analysis I want to conduct this dataframe is good to go.
I do not want to drop information now that I might need later.

In [110]:
# Saving df as csv
tweeter_df.to_csv('twitter_archive_master.csv')